## Geomorphometry II: Spatial and Temporal Terrain Analysis

Resources: [
GRASS GIS overview and manual](http://grass.osgeo.org/grass72/manuals/index.html),
[GRASSbook](http://www.grassbook.org/).


Download Mapset and color tables:

*  Download [
NagsHead time series](http://courses.ncsu.edu/mea582/common/media/01/NagsHead_series.zip) and copy it into your nc_spm_08_grass7 Location
(it should be the same level directory as PERMANENT).
Do not let your unzipping program create additional level directory with the same name!
If you are not sure about GRASS GIS Database structure read about it in
[the manual](http://grass.osgeo.org/grass72/manuals/grass_database.html).
* Custom color table for time series standard deviations map [stddev_color.txt](data/stddev_color.txt)



### Start GRASS GIS
Start GRASS - click on GRASS icon or type

In [ ]:
# using Python to initialize GRASS GIS
# This is a quick introduction into Jupyter Notebook.
# Python code can be excecuted like this:
a = 6
b = 7
c = a * b
print "Answer is", c
# Python code can be mixed with command line code (Bash).
# It is enough just to prefix the command line with an exclamation mark:
!echo "Answer is $c"
# Use Shift+Enter to execute this cell. The result is below.

In [ ]:
# using Python to initialize GRASS GIS
import os
import sys
import subprocess
from IPython.display import Image

# create GRASS GIS runtime environment
gisbase = subprocess.check_output(["grass", "--config", "path"]).strip()
os.environ['GISBASE'] = gisbase
sys.path.append(os.path.join(gisbase, "etc", "python"))

# do GRASS GIS imports
import grass.script as gs
import grass.script.setup as gsetup

# set GRASS GIS session data
rcfile = gsetup.init(gisbase, "/home/jovyan/grassdata", "nc_spm_08_grass7", "user1")

In [ ]:
# using Python to initialize GRASS GIS
# default font displays
os.environ['GRASS_FONT'] = 'sans'
# overwrite existing maps
os.environ['GRASS_OVERWRITE'] = '1'
gs.set_raise_on_error(True)
gs.set_capture_stderr(True)

In [ ]:
# using Python to initialize GRASS GIS
# set display modules to render into a file (named map.png by default)
os.environ['GRASS_RENDER_IMMEDIATE'] = 'cairo'
os.environ['GRASS_RENDER_FILE_READ'] = 'TRUE'
os.environ['GRASS_LEGEND_FILE'] = 'legend.txt'

In startup pannel set GIS Data Directory to path to datasets,
for example on MS Windows, `C:\Users\myname\grassdata`.
For Project location select nc_spm_08_grass7 (North Carolina, State Plane, meters) and
for Accessible mapset create a new mapset (called e.g. HW_terrain_analysis).
Click Start GRASS.


Change working directory:
_Settings_ > _GRASS working environment_ > _Change working directory_ > select/create any directory
or type `cd` (stands for change directory) into the GUI
_Console_ and hit Enter:

In [ ]:
# a proper directory is already set, download files
import urllib
urllib.urlretrieve("http://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/data/stddev_color.txt", "stddev_color.txt")

Download all text files (see above)
to the selected directory. Now you can use the commands from the assignment requiring the text file
without the need to specify the full path to the file.


### Compute basic topographic parameters: slope and aspect

In [ ]:
!g.region raster=elevation -p
!r.slope.aspect elevation=elevation slope=myslope aspect=myaspect

Display resulting maps with legend using GUI.

In [ ]:
!d.rast myslope
!d.legend myslope at=2,40,2,6
Image(filename="map.png")
!d.rast myaspect
!d.legend myaspect at=2,40,2,6
Image(filename="map.png")

Show impact of integer values in meters on slope and aspect pattern.
Compute integer DEM and derive its slope and aspect.
Use GUI to display the histogram: in _Map Display_ > _Analyze_ > _Create histogram_:

In [ ]:
!r.mapcalc "elev_int = int(elevation)"
!r.slope.aspect elevation=elev_int aspect=aspect_int_10m slope=slope_int_10m
!d.erase
!d.histogram slope_int_10m
!d.histogram myslope
!d.histogram aspect_int_10m
!d.histogram myaspect
!d.erase
!d.rast myslope
Image(filename="map.png")

Zoom into NW area of the current region (relatively flat area near large interchange).
Can you explain the difference in slope maps derived from integer (m vertical resolution)
and floating point (mm vertical resolution) DEMs?

In [ ]:
!d.rast slope_int_10m
Image(filename="map.png")

### Compute slope along road

Compute slope of a bus route. You will need the _myslope_ raster
map created earlier. First extract values from slope map layer:

In [ ]:
!g.region raster=elevation res=20
!v.to.rast input=busroute11 type=line output=busroute11 use=dir
!r.mapcalc "route_slope = if(busroute11, myslope)"

Then compute slope in the direction of the route:

In [ ]:
!v.to.rast input=busroute11 type=line output=busroute11_dir use=dir
!r.mapcalc "route_slope_dir = abs(atan(tan(myslope) * cos(myaspect - busroute11_dir)))"
!r.colors map=route_slope,route_slope_dir color=slope

Display, zoom if necessary, and compute univariate statistics and comment on the difference of two results.

In [ ]:
!r.univar route_slope
!r.univar route_slope_dir
!d.shade shade=elevation_shade color=myslope brighten=20
!r.contour input=elevation output=contours step=2
!d.vect contours
!d.rast route_slope
Image(filename="map.png")
!d.rast route_slope_dir
Image(filename="map.png")

### Curvatures



Compute slope, aspect and curvatures simultaneously with interpolation.
You can do the examples below for the bare earth data only (first example),
multiple return example is optional (if you are curious how it differs from BE).

In [ ]:
!g.region rural_1m  -p
!v.surf.rst input=elev_lid792_bepts elevation=elev_lid_1m aspect=asp_lid_1m pcurvature=pc_lid_1m tcurvature=tc_lid_1m npmin=120 segmax=25
!v.surf.rst input=elev_lidrural_mrpts elevation=elev_lidmr_1m aspect=asp_lidmr_1m pcurvature=pc_lidmr_1m tcurvature=tc_lidmr_1m npmin=120 segmax=25 tension=300 smooth=1.

Display the results as 2D images or in 3D view.
For 3D view, switch off everything except for elevation surface that you want to view.
Drape topographic parameters raster maps over DEMs as color.

In [ ]:
!d.erase
!d.rast elev_lid_1m
!d.rast pc_lid_1m
Image(filename="map.png")
!d.rast elev_lidmr_1m
!d.rast pc_lidmr_1m
Image(filename="map.png")

The curvature maps reflect survey pattern rather than topographic features.
So we lower the tension and increase the smoothing.
You can use multiple displays to compare the results side-by-side.

In [ ]:
!g.region rural_1m  -p
!v.surf.rst input=elev_lid792_bepts elevation=elev_lidt15_1m aspect=asp_lidt15_1m pcurvature=pc_lidt15_1m tcurvature=tc_lidt15_1m npmin=120 segmax=25 tension=15 smooth=1.
!v.surf.rst input=elev_lidrural_mrpts elevation=elev_lidmrt15_1m aspect=asp_lidmrt15_1m pcurvature=pc_lidmrt15_1m tcurvature=tc_lidmrt15_1m npmin=120 segmax=25 tension=15 smooth=1.
!d.erase
!d.rast elev_lidt15_1m
!d.rast pc_lidt15_1m
Image(filename="map.png")
!d.rast tc_lidt15_1m
!d.rast elev_lidmrt15_1m
!d.rast pc_lidmrt15_1m
Image(filename="map.png")

### Landforms


Extract landforms at different levels of detail by adjusting the size of moving window.
Set rural subregion at 1m resolution,
compute landforms using 9m and 45m neighborhood: read the manual to learn more.
Explain types of landforms and the role of the neighborhood size.

In [ ]:
!g.region rural_1m -p
!r.param.scale elev_lid792_1m output=feature9c_1m size=9  method=feature
!r.param.scale elev_lid792_1m output=feature45c_1m size=45 method=feature

Display with legend, save images for the report.
Optionally display the feature maps draped over elev_lid792_1m as color.

In [ ]:
!d.rast feature9c_1m
!d.legend feature9c_1m at=2,20,2,6
!d.rast feature45c_1m
!d.legend feature45c_1m at=2,20,2,6
!d.vect elev_lid792_cont1m color=brown
Image(filename="map.png")

### Raster time series analysis

For this exercise we will use NagsHead_series Mapset you downloaded.
You have to first make the mapset accessible.
In GUI: menu _ Settings_ -> _GRASS working environment_ -> _Mapset access_
or by using a command:

In [ ]:
!g.mapsets operation=add mapset=NagsHead_series

If you don't see the mapset, make sure you downloaded it and unzipped it correctly.


Run the series analysis and explain the results:
Which maps are core and envelope? 
Which landforms have high standard deviation and what does it mean?

In [ ]:
!g.region raster=NH_2008_1m -p
!d.erase
!d.rast NH_2008_1m
!r.series NH_1999_1m,NH_2001_1m,NH_2004_1m,NH_2005_1m,NH_2007_1m,NH_2008_1m out=NH_9908_min method=minimum
!r.series NH_1999_1m,NH_2001_1m,NH_2004_1m,NH_2005_1m,NH_2007_1m,NH_2008_1m out=NH_9908_max method=maximum
!r.series NH_1999_1m,NH_2001_1m,NH_2004_1m,NH_2005_1m,NH_2007_1m,NH_2008_1m out=NH_9908_mintime method=min_raster
!r.series NH_1999_1m,NH_2001_1m,NH_2004_1m,NH_2005_1m,NH_2007_1m,NH_2008_1m out=NH_9908_maxtime method=max_raster
!r.series NH_1999_1m,NH_2001_1m,NH_2004_1m,NH_2005_1m,NH_2007_1m,NH_2008_1m out=NH_9908_range method=range
!r.series NH_1999_1m,NH_2001_1m,NH_2004_1m,NH_2005_1m,NH_2007_1m,NH_2008_1m out=NH_9908_avg method=average
!r.series NH_1999_1m,NH_2001_1m,NH_2004_1m,NH_2005_1m,NH_2007_1m,NH_2008_1m out=NH_9908_stddev method=stddev
!r.colors NH_9908_stddev rules=stddev_color.txt
!d.rast NH_9908_stddev
!d.rast NH_9908_range
Image(filename="map.png")

Use cutting plane in 3D view to show the core and envelope.
Add constant elevation plane at -1m for reference,
set zexag somewhere 3-5 (the default is too high).
Assign surfaces constant color, use top or bottom surface for crossection color.
When using top for color, lower the light source to make
top surface dark and highlight the crossection.

In [ ]:
# end the GRASS session
os.remove(rcfile)